In [3]:
!pip install face_recognition

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566173 sha256=0b187ab4d73334eba8437d7c07fa2b5d4053d91e28293e485709596441e0f8a2
  Stored in directory: /root/.cache/pip/wheels/7a/eb/cf/e9eced74122b679557f597bb7c8e4c739cfcac526db1fd523d
Successfully built face-recognition-models


In [4]:
import time
import os
import cv2
from imutils import paths
import face_recognition
from sklearn.cluster import DBSCAN
import numpy as np
from imutils import build_montages
import collections

In [5]:
os.mkdir("/content/summerProj")
os.mkdir("/content/summerProj/videos")
os.mkdir("/content/summerProj/videos/videoFrames")
os.mkdir("/content/summerProj/static")
os.mkdir("/content/summerProj/static/faces")
os.mkdir("/content/summerProj/static/foundFaces")

In [6]:
face_cascade = cv2. CascadeClassifier (cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

In [13]:
def createFaceFrames(address,destin):
    vid = cv2.VideoCapture(address)
    count = 0
    frameCount = 0
    while vid.isOpened():
        frameCount+=1
        success,img = vid.read()
        if not success:
            break
        if frameCount%30 != 0:
            continue
        print(f"imageNum {count}")
        faces = face_cascade.detectMultiScale(img,1.3,4)
        if len(faces):
            count+=1
            cv2.imwrite(f"/content/summerProj/videos/videoFrames/{destin}_{count}.jpg",img)
    return frameCount


In [20]:
dir = '/content/summerProj/videos/videoFrames'
for f in os.listdir(dir):
    os.remove(os.path.join(dir, f))
start = time.time()
imgCount1 = createFaceFrames("/content/summerProj/videos/destinVideo.mov","destin")
end = time.time()
print(f"sample lenght: {imgCount1/30} sec")
print(f"Time for generating the frames of destinVideo: {end-start} sec")
start = time.time()
imgCount2 = createFaceFrames("/content/summerProj/videos/sourceVideo.mov","source")
end = time.time()
print(f"sample lenght: {imgCount2/30} sec")
print(f"Time for generating the frames of sourceVideo: {end-start} sec")

imageNum 0
imageNum 0
imageNum 0
imageNum 0
imageNum 1
imageNum 1
imageNum 1
imageNum 1
imageNum 2
imageNum 2
imageNum 3
imageNum 4
imageNum 4
imageNum 5
imageNum 6
imageNum 7
imageNum 7
imageNum 7
imageNum 8
imageNum 9
imageNum 10
imageNum 11
imageNum 12
imageNum 13
imageNum 14
imageNum 15
imageNum 16
imageNum 17
imageNum 17
imageNum 17
imageNum 18
imageNum 19
imageNum 20
imageNum 21
imageNum 21
imageNum 22
imageNum 22
imageNum 22
imageNum 23
imageNum 24
imageNum 25
imageNum 26
imageNum 27
imageNum 28
imageNum 29
imageNum 29
imageNum 30
imageNum 31
imageNum 32
imageNum 33
imageNum 33
imageNum 33
imageNum 34
imageNum 35
imageNum 35
imageNum 35
imageNum 36
imageNum 37
imageNum 37
imageNum 37
imageNum 37
imageNum 37
imageNum 37
imageNum 37
imageNum 37
imageNum 37
imageNum 38
imageNum 39
imageNum 39
imageNum 39
imageNum 40
imageNum 41
imageNum 41
imageNum 42
imageNum 42
imageNum 43
imageNum 44
imageNum 45
imageNum 45
imageNum 46
imageNum 46
imageNum 47
imageNum 48
imageNum 48
imageNum 48


In [21]:
print("using hog")
start = time.time()
dir = '/content/summerProj/static/faces'
for f in os.listdir(dir):
    os.remove(os.path.join(dir, f))
data = []
imagePaths = list(paths.list_images("/content/summerProj/videos/videoFrames"))
for (i, imagePath) in enumerate(imagePaths):
    print("processing image: ",i)
    image = cv2.imread(imagePath)
    rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    boxes = face_recognition.face_locations(rgb, model="hog")
    encodings = face_recognition.face_encodings(rgb, boxes)
    d = [{"imagePath": imagePath, "loc": box, "encoding": enc} for (box, enc) in zip(boxes, encodings)]
    data.extend(d)
data = np.array(data)
encodings = [d["encoding"] for d in data]
end = time.time()
print("sample size: ",len(encodings)," faces")
print("time of encoding generation:", end-start," sec")
start = time.time()
clt = DBSCAN(metric="euclidean", n_jobs=-1)
clt.fit(encodings)
end = time.time()
print("time of clustering:", end-start," sec")
labels = clt.labels_
labelIDs = np.unique(labels)

using hog
processing image:  0
processing image:  1
processing image:  2
processing image:  3
processing image:  4
processing image:  5
processing image:  6
processing image:  7
processing image:  8
processing image:  9
processing image:  10
processing image:  11
processing image:  12
processing image:  13
processing image:  14
processing image:  15
processing image:  16
processing image:  17
processing image:  18
processing image:  19
processing image:  20
processing image:  21
processing image:  22
processing image:  23
processing image:  24
processing image:  25
processing image:  26
processing image:  27
processing image:  28
processing image:  29
processing image:  30
processing image:  31
processing image:  32
processing image:  33
processing image:  34
processing image:  35
processing image:  36
processing image:  37
processing image:  38
processing image:  39
processing image:  40
processing image:  41
processing image:  42
processing image:  43
processing image:  44
processing

In [22]:
count = 0
for i in labels:
  if i <0:
    count+=1
print(f"Total unclasifiable faces : {count}")
print("Percentage of unclassified samples: ",100*count/len(labels),"%")
print("Classifying Efficiency: ",100-(100*count/len(labels)),"%")

Total unclasifiable faces : 77
Percentage of unclassified samples:  28.94736842105263 %
Classifying Efficiency:  71.05263157894737 %


In [23]:
for labelID in labelIDs:
        # if labelID<0:
            # continue
        allIdxs = np.where(labels == labelID)[0]
        idxs = np.random.choice(allIdxs, size=min(25, len(allIdxs)),
    replace=False)
        faces = []
        for i in idxs:
            imgpath = data[i]["imagePath"]
            temp = imgpath.split("/")[-1]
            imgtype = temp.split("_")[0]
            image = cv2.imread(data[i]["imagePath"])
            (top, right, bottom, left) = data[i]["loc"]
            face = image[top:bottom, left:right]
            face = cv2.resize(face, (96, 96))
            faces.append(face)
        montage = build_montages(faces, (96, 96), (5, 5))[0]
        cv2.imwrite(f"/content/summerProj/static/faces/{labelID}.jpg",montage)




In [50]:
print(labelIDs)

[-1  0  1  2  3  4  5  6  7  8  9 10 11 12 13]


In [24]:
print(labels)

[ 0  1 -1  9  2 -1  3  4  4  3  5 -1  0 -1  5  0  9 -1  6 -1  0  1 -1  0
  7  2  8  0  3 -1  0 -1  1  0 -1  9  7  8 12  6 10  0  5 -1 11 -1 -1  9
  7  8  6 -1  5 -1 -1  1  6  5  4  3  7  0  7  0  0  7 -1 -1  4  3  7  0
 -1  3  4  0  1 -1 -1  7  3  4  0  7  0  7 11  0  1 -1  9  6  0  4  3  0
  3  4  9 -1 -1  0  7 10  6 -1 -1  0  0  2  7  9 12 -1 11 -1 -1 11 -1  0
  7  7  9  0 -1  1  7  0 -1 10  7  8  6 12  2  0  0  2 10  3  4  0  1  7
  0 -1  7  0  7  7  0  2  6  5  2 -1  0  9 -1 12  0  7  9  0  1 -1 -1  0
 -1 -1  7 -1 -1  3  4  7  8  9 10  0  3  4  4  3  4  3 -1 -1  0  5  0  9
 -1  6 -1  1  0 -1 -1  0  1  5 -1 -1  5 -1 -1 -1  0 -1 -1  4  3  3  4  5
  7  0  0  6  0 11 -1 -1  0 -1  9 -1 -1  7 -1  0 -1 -1 -1 -1 -1 -1  1  7
 -1 -1  2  7  6 10  0  8 12  7 -1  0  1 -1 -1  1 -1  0  0 -1 -1 -1  0  0
 -1 -1]


In [25]:
print(np.shape(encodings))
print("here first number is the number of faces and second number is\nthe number of dimensions of the encoding data")
print("Like for 2d data there would be only two x,y and for 3d there would be x,y,z\nsimilarily there are 128 dimensions")

(266, 128)
here first number is the number of faces and second number is
the number of dimensions of the encoding data
Like for 2d data there would be only two x,y and for 3d there would be x,y,z
similarily there are 128 dimensions
